In [1]:
import os
from os import listdir
from os.path import join, isfile
from PIL import Image

import numpy as np
import pandas as pd
# import tensorflow as tf
import matplotlib.pyplot as plt
import cv2

import torch
import torchvision
import torch.nn.functional as F
from torch import nn, optim
from efficientnet_pytorch import EfficientNet
from torchvision.transforms import transforms, Lambda
from torch.utils.data import random_split
import random
from torch.utils.data import DataLoader
import glob

In [2]:
original_address = '/scratch/student/shokoofa/complete_dataset/'

# '/scratch/student/shokoofa/K-fold Experiments (copy)/fold 2/dset/'

In [3]:
# train_path = set('/'.join(i.split('/')[-2:]) for i in glob.glob(os.path.join(original_address + 'Train/*', '*')))
# valid_path = set('/'.join(i.split('/')[-2:]) for i in glob.glob(os.path.join(original_address + 'Val/*', '*')) )
# test_path = set('/'.join(i.split('/')[-2:]) for i in glob.glob(os.path.join(original_address + 'Test/*', '*')) )

# info = pd.read_csv(os.path.join(original_address , 'new_labels.csv'))

# info['sample_path'] = info['sample_path'].astype(str)
# info['path'] = info['sample_path'].apply(lambda x: '/'.join(x.split(' ')[:2]))
# label_path = info['path'].values
# data_labels = info['label'].values

# image_train_path = []
# image_test_path = []
# image_val_path = []

# label_train_path = []
# label_test_path = []
# label_val_path = []

# for i,sub_folder in enumerate(label_path):
#     if sub_folder in train_path:
#         label_train_path.append(data_labels[i])
#         image_train_path.append('Train/' + sub_folder)
#     elif sub_folder in valid_path:
#         label_val_path.append(data_labels[i])
#         image_val_path.append('Valid/' + sub_folder)
#     elif sub_folder in test_path:
#         label_test_path.append(data_labels[i])
#         image_test_path.append('Test/' + sub_folder)

In [4]:
train_df = pd.read_csv('/scratch/student/shokoofa/video analysis/Fold 1/csv_analysis/dset_files/train_label_original.csv')
val_df = pd.read_csv('/scratch/student/shokoofa/video analysis/Fold 1/csv_analysis/dset_files/val_label_original.csv')
test_df = pd.read_csv('/scratch/student/shokoofa/video analysis/Fold 1/csv_analysis/dset_files/test_label_original.csv')

# train_df.to_csv('new-dataset/train_labels_original2.csv', index=False)
# val_df.to_csv('new-dataset/val_labels_original2.csv', index=False)
# test_df.to_csv('new-dataset/test_labels_original2.csv', index=False)

In [5]:
train_df

,label,path
0,Subcostal,Train/_3wRyUbdSGmjEWULRVg-jw/3/0
1,Subcostal,Train/_3wRyUbdSGmjEWULRVg-jw/3/1
2,Subcostal,Train/_3wRyUbdSGmjEWULRVg-jw/4/0
3,Subcostal,Train/_3wRyUbdSGmjEWULRVg-jw/4/1
4,Subcostal,Train/_3wRyUbdSGmjEWULRVg-jw/5/0
...,...,...
5096,Subcostal,Train/6VLObFIMTzq7fyUz9-ltQA/77/1
5097,Suprasternal,Train/6VLObFIMTzq7fyUz9-ltQA/78/0
5098,Suprasternal,Train/6VLObFIMTzq7fyUz9-ltQA/78/1
5099,Suprasternal,Train/6VLObFIMTzq7fyUz9-ltQA/79/0


In [6]:
test_df

,label,path
0,PLAX,Test/_497VMYVSZywku-h2UmnUw/2/0
1,PLAX,Test/_497VMYVSZywku-h2UmnUw/3/0
2,PLAX,Test/_497VMYVSZywku-h2UmnUw/3/1
3,PLAX,Test/_497VMYVSZywku-h2UmnUw/4/0
4,PLAX,Test/_497VMYVSZywku-h2UmnUw/4/1
...,...,...
600,Subcostal,Test/6rn5hZKmRLaqZAYRjc2s6g/62/1
601,Subcostal,Test/6rn5hZKmRLaqZAYRjc2s6g/63/0
602,Subcostal,Test/6rn5hZKmRLaqZAYRjc2s6g/63/1
603,Suprasternal,Test/6rn5hZKmRLaqZAYRjc2s6g/65/0


In [7]:
def creating_new_df(path_list, label_list, dict, window_size, window_slide, set_name):
    for idx in range(len(path_list)):
        frames_per_sample = []
        for item in glob.glob(os.path.join(original_address + path_list[idx], '*')):
            frames_per_sample.append(int(item.split('_')[-1].replace('.jpeg', '')))
            # print(frames_per_sample)
        frames_per_sample = sorted(frames_per_sample)
        # print(image_list[idx])
        length = frames_per_sample[-1] - frames_per_sample[0]
        i = 0
        ls = []
        while (i+ window_size <= length):
            ls.append([*range(frames_per_sample[i], frames_per_sample[i + window_size])])
            i += window_slide
        for num_sub_videos in range(len(ls)):
            # print('/'.join(item.split('/')[-4:-1]))
            dict['path'].append(set_name + '/' + '/'.join(item.split('/')[-4:-1])) 
            dict['label'].append(label_list[idx])
            dict['start'].append(ls[num_sub_videos][0])
            dict['end'].append(ls[num_sub_videos][-1])
    # new_df = pd.DataFrame(dict)
    return pd.DataFrame(dict)

In [8]:
window_size = 16
window_slide = 2
dict_tr = {'path':[], 'label':[], 'start': [], 'end': []}
new_train_df = creating_new_df(train_df['path'], train_df['label'], dict_tr, window_size=window_size, window_slide=window_slide, set_name = 'Train' )

In [9]:
dict_val = {'path':[], 'label':[], 'start': [], 'end': []}
new_val_df = creating_new_df(val_df['path'], val_df['label'], dict_val,  window_size=window_size, window_slide=window_slide, set_name = 'Val'  )

In [10]:
dict_test = {'path':[], 'label':[], 'start': [], 'end': []}
new_test_df = creating_new_df(test_df['path'], test_df['label'], dict_test,  window_size=window_size, window_slide=window_slide, set_name = 'Test'  )

In [12]:
new_test_df

,path,label,start,end
0,Test/_497VMYVSZywku-h2UmnUw/2/0,PLAX,1,16
1,Test/_497VMYVSZywku-h2UmnUw/2/0,PLAX,3,18
2,Test/_497VMYVSZywku-h2UmnUw/2/0,PLAX,5,20
3,Test/_497VMYVSZywku-h2UmnUw/2/0,PLAX,7,22
4,Test/_497VMYVSZywku-h2UmnUw/2/0,PLAX,9,24
...,...,...,...,...
6862,Test/6rn5hZKmRLaqZAYRjc2s6g/65/1,Suprasternal,81,96
6863,Test/6rn5hZKmRLaqZAYRjc2s6g/65/1,Suprasternal,83,98
6864,Test/6rn5hZKmRLaqZAYRjc2s6g/65/1,Suprasternal,85,100
6865,Test/6rn5hZKmRLaqZAYRjc2s6g/65/1,Suprasternal,87,102


In [79]:
# def combine_psax_views(new_df):
#     apical_indx = new_df[new_train_df['label']== 'PSAX-apical'].index
#     new_df.loc[apical_indx, 'label'] = 'PSAX-mid'
#     new_df = new_df[new_df['label'] != 'PSAX-apical']
#     return new_df

In [80]:
# new_train_df = combine_psax_views(new_train_df)
# new_test_df= combine_psax_views(new_test_df)
# new_val_df = combine_psax_views(new_val_df)

/tmp/ipykernel_4019003/2237947100.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  apical_indx = new_df[new_train_df['label']== 'PSAX-apical'].index


In [11]:
new_val_df.label.unique()

array(['PLAX', 'PSAX-ves', 'PSAX-base', 'PSAX-mid', 'PSAX-apical',
       'Apical-4ch', 'Apical-5ch', 'Apical-2ch', 'Apical-3ch',
       'Subcostal', 'Suprasternal'], dtype=object)

In [76]:
# # ### for big general datast 
# df_final = pd.concat([new_train_df, new_val_df, new_test_df], axis = 0)
# df_final.to_csv('final_large_labels.csv', index= False)

In [13]:
new_train_df.to_csv('dset_files/train_label_sw16.csv', index=False)
new_val_df.to_csv('dset_files/val_label_sw16.csv', index=False)
new_test_df.to_csv('dset_files/test_label_sw16.csv', index=False)